In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# spark = SparkSession.builder \
#     .appName("Kafka to Minio") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("Kafka to Minio") \
    .config("spark.task.maxFailures", "1") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.s3a.access.key", "wFw2nWmmNHNivuqh") \
    .config("spark.hadoop.fs.s3a.secret.key", "Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.ssl.enabled", "false") \
    .getOrCreate()
# tiAoMHTLqpoi7TKMVXQGPR7N0VT6dRsJ
# E0QTNG3vCGYwqpar

spark.sparkContext.setLogLevel('INFO')
# spark.sparkContext.setLogLevel('ERROR')

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-broker:9092") \
    .option("subscribe", "user") \
    .load()

df = df.selectExpr("CAST(value AS STRING)")

query = df \
    .writeStream \
    .format("json") \
    .option("path", "s3a://raw/user_spark") \
    .option("checkpointLocation", "/tmp/checkpoints") \
    .trigger(processingTime='1 seconds') \
    .start()

# /tmp/checkpoints

query.awaitTermination()

Py4JJavaError: An error occurred while calling o69.start.
: org.apache.hadoop.fs.s3a.AWSClientIOException: getFileStatus on s3a://raw/user_spark/_spark_metadata: com.amazonaws.SdkClientException: Unable to execute HTTP request: Unsupported or unrecognized SSL message: Unable to execute HTTP request: Unsupported or unrecognized SSL message
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:208)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3289)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3185)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3053)
	at org.apache.hadoop.fs.DelegateToFileSystem.getFileStatus(DelegateToFileSystem.java:128)
	at org.apache.hadoop.fs.FileContext$15.next(FileContext.java:1247)
	at org.apache.hadoop.fs.FileContext$15.next(FileContext.java:1243)
	at org.apache.hadoop.fs.FS